This notebook analyzes cases deemed medically qualified that went to the PRB for a vote from Jan. 1, 2022 to Jun. 16, 2023.

In [57]:
import pandas as pd
import re

In [14]:
# load source data copied down from google sheets

# load grants
grants = pd.read_csv('source/prb_mr_case_voting_tracker_grants_6.16.23.csv')

# load denials
den = pd.read_csv('source/prb_mr_case_voting_tracker_denials_6.16.23.csv')

In [16]:
grants.head()

,MR #,Last Name,First Name,IDOC #,Date granted,Vote,Members
0,MR0009,Bahrs,Shawn,B19308,4/18/2022,(3-0),"For: Crigler, Bohland Miller"
1,MR0105,Barnett,C A,Y20852,3/24/2023,(3-0),"For: Coates, Shelton, Shoffner"
2,MR0097,Belmore,Bryant,N12623,3/3/2023,(3-0),"For: Crigler, Globokar, Miller"
3,MR0071,Brookmyer,Rita,N97463,11/18/2022,(3-0),"For: Globokar, Shoffner, Tupy"
4,MR0047,Burke,Dewayne,N80531,7/29/2022,(3-0),"For: Crigler, Bohland, Shelton"


In [17]:
den.head()

,MR #,Last Name,First Name,IDOC #,Date denied,Vote,Members
0,MR0002,Burke,Dwayne,N80531,3/16/2022,(3-0),"For: Bohland, Miller, Tupy"
1,MR0013,Miller,Joseph,A82547,4/18/2022,(3-0),"For: Crigler, Bohland Miller"
2,MR0014,Perkins,Ernest,N82771,4/18/2022,(3-0),"For: Crigler, Bohland Miller"
3,MR0015,Wiggins,Malcolm,N04370,4/18/2022,(3-0),"For: Crigler, Bohland Miller"
4,MR0011,Rhoiney,Eugene,A15243,4/18/2022,(3-0),"For: Crigler, Bohland Miller"


In [18]:
# prep one combined cases df

# rename date cols
grants = grants.rename(columns={'Date granted': 'Date decided'})
den = den.rename(columns={'Date denied': 'Date decided'})

# add row for decision and append grants and denials
grants['decision'] = 'grant'
den['decision'] = 'denial'

cases = pd.concat([grants, den])

In [25]:
# clean data types

# convert date granted to datetime object
cases['Date decided'] = pd.to_datetime(cases['Date decided'])

# create a year col
cases['year'] = cases['Date decided'].dt.year

# create a month_year col
cases['month_year'] = cases['Date decided'].dt.strftime('%m/%Y')

# create a month col
cases['month'] = cases['Date decided'].dt.month

# format members and votes as strings
cases['Members'] = cases['Members'].astype(str)
cases['Vote'] = cases['Vote'].astype(str)

In [221]:
cases.head()

,MR #,Last Name,First Name,IDOC #,Date decided,Vote,Members,decision,year,month_year,month,members_for,members_against,panel
0,MR0009,Bahrs,Shawn,B19308,2022-04-18,(3-0),"For: Crigler, Bohland Miller",grant,2022,04/2022,4,Crigler Bohland Miller,,Crigler Bohland Miller
1,MR0105,Barnett,C A,Y20852,2023-03-24,(3-0),"For: Coates, Shelton, Shoffner",grant,2023,03/2023,3,Coates Shelton Shoffner,,Coates Shelton Shoffner
2,MR0097,Belmore,Bryant,N12623,2023-03-03,(3-0),"For: Crigler, Globokar, Miller",grant,2023,03/2023,3,Crigler Globokar Miller,,Crigler Globokar Miller
3,MR0071,Brookmyer,Rita,N97463,2022-11-18,(3-0),"For: Globokar, Shoffner, Tupy",grant,2022,11/2022,11,Globokar Shoffner Tupy,,Globokar Shoffner Tupy
4,MR0047,Burke,Dewayne,N80531,2022-07-29,(3-0),"For: Crigler, Bohland, Shelton",grant,2022,07/2022,7,Crigler Bohland Shelton,,Crigler Bohland Shelton


In [126]:
# clean up and reformat Members col int members_for and members_against cols

# create functions to grab for and against members from the Members 
def get_members_for(row):
    split_list = re.split('Against', row[6])
    members_for = split_list[0].replace('For: ', '')
    return members_for

def get_members_against(row):
    split_list = re.split('Against', row[6])
    if len(split_list) == 2:
        return split_list[1]
    else:
        return ''
    
# apply these functions to cases in two new cols: members_for and members_against
cases['members_for'] = cases.apply(get_members_for, axis=1)
cases['members_against'] = cases.apply(get_members_against, axis=1)

In [134]:
cases.tail()

,MR #,Last Name,First Name,IDOC #,Date decided,Vote,Members,decision,year,month_year,month,members_for,members_against
70,MR0128,Linzy,Joe,B03244,2023-06-02,(3-0),"For: Miller, Shoffner, Grubbs",denial,2023,06/2023,6,"Miller, Shoffner, Grubbs",
71,MR0123,Hughes,Stanley,A60835,2023-06-16,(3-0),"For: Bohland, Globokar, Terrones",denial,2023,06/2023,6,"Bohland, Globokar, Terrones",
72,MR0132,Green,Tyreed,A01806,2023-06-16,(3-0),"For: Bohland, Globokar, Terrones",denial,2023,06/2023,6,"Bohland, Globokar, Terrones",
73,MR0124,Barnwell,James,A15031,2023-06-16,(3-0),"For: Bohland, Globokar, Terrones",denial,2023,06/2023,6,"Bohland, Globokar, Terrones",
74,MR0129,Hrecko,Matthew,A76273,2023-06-16,(2-1),"For: Bohland Terrones, Against Globokar",denial,2023,06/2023,6,"Bohland Terrones,",Globokar


In [176]:
# get panel name col

# create a function to clean the string of members names
def clean(string):
    string = string.replace('.','')
    string = string.replace(':','')
    string = string.replace(',','')
    string = string.strip()
    return string

# apply the function to clean up members_for and members_against cols
cases['members_for'] = cases['members_for'].apply(clean)
cases['members_against'] = cases['members_against'].apply(clean)

# create a panel col
cases['panel'] = cases['members_for'] + ' ' + cases['members_against']

In [210]:
# import manually created master list of prb members: current and former and their affiliations
prb = pd.read_csv('manual/PRB Board Members Master List (Current and Former) - Master List_8.16.23.csv')

In [211]:
prb.head()

,Name,Last,First,Member Expiration,Member Since,Affiliation,Appointed By,Position,County
0,"Bohland, Jared",Bohland,Jared,01/18/2027,05/03/2021,Republican,Governor,Member,Macon
1,"Coates, Matthew",Coates,Matthew,01/18/2027,07/01/2022,Democrat,Governor,Member,McLean
2,"Globokar, Julie",Globokar,Julie,01/20/2025,07/05/2022,Democrat,Governor,Member,Cook
3,"Goff, Darryldean",Goff,Darryldean,01/20/2025,05/15/2023,Democrat,Governor,Member,Lawrence
4,"Grubbs, Jeffrey",Grubbs,Jeffrey,01/20/2025,11/01/2022,Democrat,Governor,Member,Jackson


In [219]:
# create a new dataframe by member with yays and nays

# convert Last to string
prb['Last'] = prb['Last'].astype(str)

# functions to parse cases df and look up the number of instances each member shows up in the members_for and members_against cols
def get_yays(member):
    yays = cases['members_for'].str.count(member)
    return yays.sum()

def get_nays(member):
    nays = cases['members_against'].str.count(member)
    return nays.sum()

prb['yays'] = prb['Last'].apply(get_yays)
prb['nays'] = prb['Last'].apply(get_nays)

In [220]:
prb

,Name,Last,First,Member Expiration,Member Since,Affiliation,Appointed By,Position,County,yays,nays
0,"Bohland, Jared",Bohland,Jared,01/18/2027,05/03/2021,Republican,Governor,Member,Macon,44,8
1,"Coates, Matthew",Coates,Matthew,01/18/2027,07/01/2022,Democrat,Governor,Member,McLean,16,0
2,"Globokar, Julie",Globokar,Julie,01/20/2025,07/05/2022,Democrat,Governor,Member,Cook,22,3
3,"Goff, Darryldean",Goff,Darryldean,01/20/2025,05/15/2023,Democrat,Governor,Member,Lawrence,0,0
4,"Grubbs, Jeffrey",Grubbs,Jeffrey,01/20/2025,11/01/2022,Democrat,Governor,Member,Jackson,13,0
5,"Heaton, Rodger",Heaton,Rodger,01/20/2025,05/01/2022,Republican,Governor,Member,Cook,20,1
6,"Miller, Lee Ann",Miller,Lee Ann,01/18/2027,09/02/2021,Republican,Governor,Member,Gallatin,54,2
7,"Shelton, Donald",Shelton,Donald,01/15/2029,09/20/2012,Republican,Governor,Chair,Sangamon,37,0
8,"Shoffner, Robin",Shoffner,Robin,01/18/2027,05/01/2022,Democrat,Governor,Member,Cook,27,0
9,"Terrones, Carmen",Terrones,Carmen,01/20/2025,06/27/2022,Democrat,Governor,Member,Cook,27,0


In [218]:
# test
cases.to_csv('test.csv')

In [177]:
cases

,MR #,Last Name,First Name,IDOC #,Date decided,Vote,Members,decision,year,month_year,month,members_for,members_against,panel
0,MR0009,Bahrs,Shawn,B19308,2022-04-18,(3-0),"For: Crigler, Bohland Miller",grant,2022,04/2022,4,Crigler Bohland Miller,,Crigler Bohland Miller
1,MR0105,Barnett,C A,Y20852,2023-03-24,(3-0),"For: Coates, Shelton, Shoffner",grant,2023,03/2023,3,Coates Shelton Shoffner,,Coates Shelton Shoffner
2,MR0097,Belmore,Bryant,N12623,2023-03-03,(3-0),"For: Crigler, Globokar, Miller",grant,2023,03/2023,3,Crigler Globokar Miller,,Crigler Globokar Miller
3,MR0071,Brookmyer,Rita,N97463,2022-11-18,(3-0),"For: Globokar, Shoffner, Tupy",grant,2022,11/2022,11,Globokar Shoffner Tupy,,Globokar Shoffner Tupy
4,MR0047,Burke,Dewayne,N80531,2022-07-29,(3-0),"For: Crigler, Bohland, Shelton",grant,2022,07/2022,7,Crigler Bohland Shelton,,Crigler Bohland Shelton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,MR0128,Linzy,Joe,B03244,2023-06-02,(3-0),"For: Miller, Shoffner, Grubbs",denial,2023,06/2023,6,Miller Shoffner Grubbs,,Miller Shoffner Grubbs
71,MR0123,Hughes,Stanley,A60835,2023-06-16,(3-0),"For: Bohland, Globokar, Terrones",denial,2023,06/2023,6,Bohland Globokar Terrones,,Bohland Globokar Terrones
72,MR0132,Green,Tyreed,A01806,2023-06-16,(3-0),"For: Bohland, Globokar, Terrones",denial,2023,06/2023,6,Bohland Globokar Terrones,,Bohland Globokar Terrones
73,MR0124,Barnwell,James,A15031,2023-06-16,(3-0),"For: Bohland, Globokar, Terrones",denial,2023,06/2023,6,Bohland Globokar Terrones,,Bohland Globokar Terrones


In [178]:
test = cases.head(10)
test

,MR #,Last Name,First Name,IDOC #,Date decided,Vote,Members,decision,year,month_year,month,members_for,members_against,panel
0,MR0009,Bahrs,Shawn,B19308,2022-04-18,(3-0),"For: Crigler, Bohland Miller",grant,2022,04/2022,4,Crigler Bohland Miller,,Crigler Bohland Miller
1,MR0105,Barnett,C A,Y20852,2023-03-24,(3-0),"For: Coates, Shelton, Shoffner",grant,2023,03/2023,3,Coates Shelton Shoffner,,Coates Shelton Shoffner
2,MR0097,Belmore,Bryant,N12623,2023-03-03,(3-0),"For: Crigler, Globokar, Miller",grant,2023,03/2023,3,Crigler Globokar Miller,,Crigler Globokar Miller
3,MR0071,Brookmyer,Rita,N97463,2022-11-18,(3-0),"For: Globokar, Shoffner, Tupy",grant,2022,11/2022,11,Globokar Shoffner Tupy,,Globokar Shoffner Tupy
4,MR0047,Burke,Dewayne,N80531,2022-07-29,(3-0),"For: Crigler, Bohland, Shelton",grant,2022,07/2022,7,Crigler Bohland Shelton,,Crigler Bohland Shelton
5,MR0104,Burns,Richard,R31862,2023-03-24,(3-0),"For: Coates, Shelton, Shoffner",grant,2023,03/2023,3,Coates Shelton Shoffner,,Coates Shelton Shoffner
6,MR0098,Castillo,Martin,M26091,2023-03-03,(3-0),"For: Crigler, Globokar, Miller",grant,2023,03/2023,3,Crigler Globokar Miller,,Crigler Globokar Miller
7,MR0121,Chairs,Amos,A90420,2023-06-02,(3-0),"For: Miller, Shoffner, Grubbs",grant,2023,06/2023,6,Miller Shoffner Grubbs,,Miller Shoffner Grubbs
8,MR0086,Cody,Lawrence,K039605,2023-01-20,(2-1),"For: Heaton, Shoffner. Against: Bohland",grant,2023,01/2023,1,Heaton Shoffner,Bohland,Heaton Shoffner Bohland
9,MR0126,Colbert,Saul,A15110,2023-06-16,(2-1),"For: Globokar, Terrones Against Bohland",grant,2023,06/2023,6,Globokar Terrones,Bohland,Globokar Terrones Bohland


In [182]:
a = test['members_for'].str.count('Crigler')
type(a)
a

0    1
1    0
2    1
3    0
4    1
5    0
6    1
7    0
8    0
9    0
Name: members_for, dtype: int64

In [183]:
a.sum()

4

In [184]:
prb_members_list = ['Crigler', 'Bohland', 'Miller']

In [195]:
prb_votes_dict = {}
prb_votes_dict['member'] = []
prb_votes_dict['yays'] = []
prb_votes_dict['nays'] = []

for member in prb_members_list:
    # add member name to dict
    prb_votes_dict['member'].append(member)
    
    # count yays in members_for col
    yays = test['members_for'].str.count(member)
    prb_votes_dict['yays'].append(yays.sum())
    
    # count nays
    nays = test['members_against'].str.count(member)
    prb_votes_dict['nays'].append(nays.sum())

In [196]:
prb_votes_dict

{'member': ['Crigler', 'Bohland', 'Miller'],
 'yays': [4, 2, 4],
 'nays': [0, 2, 0]}

In [197]:
df = pd.DataFrame.from_dict(prb_votes_dict) 

In [198]:
df

,member,yays,nays
0,Crigler,4,0
1,Bohland,2,2
2,Miller,4,0


## FINDING: approvals/denials by month and year

In [50]:
# pivot by month_year and decision outcome
cases_by_month = pd.pivot_table(cases, values='MR #', index='month_year', columns='decision', aggfunc='count').reset_index()

In [51]:
# fill NaN with 0
cases_by_month = cases_by_month.fillna(0)

In [52]:
# clean up pivoted data

# fill NaN with 0
cases_by_month = cases_by_month.fillna(0)

# format as date
cases_by_month['month_year'] = pd.to_datetime(cases_by_month['month_year'])

# reformat as mm/yyy
# cases_by_month['month_year'] = cases_by_month['month_year'].dt.strftime('%m/%Y')

In [32]:
# check denial count is 75
cases_by_month['denial'].sum()

75.0

In [33]:
# check grant count is 46
cases_by_month['grant'].sum()

46.0

In [53]:
# copied this into findings doc
cases_by_month.sort_values('month_year', ascending=True)

decision,month_year,denial,grant
1,2022-02-01,0.0,2.0
3,2022-03-01,1.0,3.0
5,2022-04-01,11.0,1.0
7,2022-05-01,8.0,2.0
9,2022-06-01,6.0,1.0
11,2022-07-01,6.0,4.0
12,2022-08-01,0.0,2.0
13,2022-09-01,3.0,5.0
14,2022-10-01,6.0,3.0
15,2022-11-01,4.0,2.0
